# Hyper parameter tunning



In [17]:
import pandas as pd 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
import pickle

In [39]:
from sklearn.model_selection import train_test_split

In [18]:
Train = pd.read_excel('train_data.xlsx')
train = Train[['idf_vector', 'Sarcasm']]

#X_train = train['idf_vector']
#y_train = train['Sarcasm']

In [19]:
Test = pd.read_excel('test_data.xlsx')
test = Test[['idf_vector', 'Sarcasm']]
#X_test = test['idf_vector']
#y_test = test['Sarcasm']

In [20]:

# Save the array to a file
with open('train.pkl', 'wb') as file:
    pickle.dump(train, file)
# Save the array to a file
with open('test.pkl', 'wb') as file:
    pickle.dump(test, file)


In [21]:
# Load training data from pickle
with open('train.pkl', 'rb') as file:
    train_pk = pickle.load(file)

# Load test data from pickle
with open('test.pkl', 'rb') as file:
    test_pk = pickle.load(file)


In [22]:
train_pk

,idf_vector,Sarcasm
0,[0.35064284 0. 0.70874616 0.35064284 0...,0
1,[0.36828071 0. 0.74407735 0.36828071 0...,1
2,[0.33881107 0.00515968 0.68165561 0.33881107 0...,0
3,[0.3352069 0. 0.68956848 0.3352069 0...,0
4,[0.32067146 0. 0.7215108 0.32067146 0...,1
...,...,...
5192,[0.33714827 0. 0.73559621 0.33714827 0...,1
5193,[0.30734299 0. 0.68298441 0.30734299 0...,1
5194,[0.29527348 0. 0.68897146 0.29527348 0...,1
5195,[0.31039716 0. 0.74495319 0.31039716 0...,1


In [23]:
test_pk

,idf_vector,Sarcasm
0,[0.36049631 0. 0.7245443 0.36049631 0...,0
1,[0.34694305 0. 0.69611009 0.34694305 0...,0
2,[0.32716079 0. 0.70465401 0.32716079 0...,1
3,[0.32695773 0. 0.7006237 0.32695773 0...,1
4,[0.2973631 0. 0.65419881 0.2973631 0...,0
...,...,...
1295,[0.35472576 0. 0.71249637 0.35472576 0...,0
1296,[0.35571375 0. 0.71469093 0.35571375 0...,0
1297,[0.32585507 0.08336784 0.71688115 0.32585507 0...,1
1298,[0.32020593 0. 0.69863111 0.32020593 0...,1


In [28]:
train_pk['idf_vector'] = train_pk['idf_vector'].apply(lambda x: np.fromstring(x.replace('\n', '')[1:-1], sep=' '))
test_pk['idf_vector'] = test_pk['idf_vector'].apply(lambda x: np.fromstring(x.replace('\n', '')[1:-1], sep=' '))

# Split into X_train and y_train
X_train = np.vstack(train_pk['idf_vector'].values)
y_train = train_pk['Sarcasm']

X_test = np.vstack(test_pk['idf_vector'].values)
y_test = test_pk['Sarcasm']

In [29]:
def logistic_regression(X_train, X_test, y_train, y_test, parameters):
    # Perform grid search for hyperparameter tuning
    log_reg = LogisticRegression()
    grid_search = GridSearchCV(log_reg, parameters, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    
    # Extract best parameters
    best_params = grid_search.best_params_
    print(f"Best parameters found: {best_params}")
    
    # Train model with best parameters
    lr_model = LogisticRegression(**best_params)
    lr_model.fit(X_train, y_train)
    lr_preds = lr_model.predict(X_test)
    
    # Evaluation
    accuracy = accuracy_score(y_test, lr_preds)
    conf_matrix = confusion_matrix(y_test, lr_preds)
    class_report = classification_report(y_test, lr_preds)

    print(f"Evaluation for the given vectors:\n")
    print(f'Accuracy: {accuracy:.2f}')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('Classification Report:')
    print(class_report)
    
    return lr_model


In [30]:
#hyper parameters
log_parameters = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs'],
    'max_iter': [100, 200, 500, 1000]
}
log_model=logistic_regression(X_train,X_test, y_train, y_test,log_parameters)


C:\Users\LENOVO\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\LENOVO\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

Best parameters found: {'C': 10, 'max_iter': 100, 'solver': 'liblinear'}
Evaluation for the given vectors:

Accuracy: 0.76
Confusion Matrix:
[[494 110]
 [207 489]]
Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.82      0.76       604
           1       0.82      0.70      0.76       696

    accuracy                           0.76      1300
   macro avg       0.76      0.76      0.76      1300
weighted avg       0.76      0.76      0.76      1300



Best parameters found: {'C': 10, 'max_iter': 100, 'solver': 'liblinear'}
Evaluation for the given vectors:

Accuracy: 0.76
Confusion Matrix:
[[494 110]
 [207 489]]
Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.82      0.76       604
           1       0.82      0.70      0.76       696

    accuracy                           0.76      1300
   macro avg       0.76      0.76      0.76      1300
weighted avg       0.76      0.76      0.76      1300


In [31]:
def svm(X_train, X_test, y_train, y_test, parameters):
    # Perform grid search for hyperparameter tuning
    svm_model = SVC()
    grid_search = GridSearchCV(svm_model, parameters, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    
    # Extract best parameters
    best_params = grid_search.best_params_
    print(f"Best parameters found: {best_params}")
    
    # Train model with best parameters
    svm_model = SVC(**best_params)
    svm_model.fit(X_train, y_train)
    svm_preds = svm_model.predict(X_test)
    
    # Evaluation
    accuracy = accuracy_score(y_test, svm_preds)
    conf_matrix = confusion_matrix(y_test, svm_preds)
    class_report = classification_report(y_test, svm_preds)

    print(f"Evaluation for the given vectors:\n")
    print(f'Accuracy: {accuracy:.2f}')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('Classification Report:')
    print(class_report)
    
    return svm_model

In [33]:
#hyper Parameters
svm_parameters = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

model_svm = svm(X_train, X_test, y_train, y_test, svm_parameters)

Best parameters found: {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}
Evaluation for the given vectors:

Accuracy: 0.80
Confusion Matrix:
[[531  73]
 [193 503]]
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.88      0.80       604
           1       0.87      0.72      0.79       696

    accuracy                           0.80      1300
   macro avg       0.80      0.80      0.80      1300
weighted avg       0.81      0.80      0.79      1300



Best parameters found: {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}
Evaluation for the given vectors:

Accuracy: 0.80
Confusion Matrix:
[[531  73]
 [193 503]]
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.88      0.80       604
           1       0.87      0.72      0.79       696

    accuracy                           0.80      1300
   macro avg       0.80      0.80      0.80      1300
weighted avg       0.81      0.80      0.79      1300



In [44]:
def random_forest(X_train, X_test, y_train, y_test,parameters):
    # Perform grid search for hyperparameter tuning
    X_t,_,y_t,_=train_test_split(X_train, y_train, train_size=0.1, random_state=42)
    rf_model = RandomForestClassifier()
    grid_search = GridSearchCV(rf_model, parameters, cv=5, scoring='accuracy')
    grid_search.fit(X_t, y_t)
    
    # Extract best parameters
    best_params = grid_search.best_params_
    print(f"Best parameters found: {best_params}")
    
    # Train model with best parameters
    rf_model = RandomForestClassifier(**best_params)
    rf_model.fit(X_train, y_train)
    rf_preds = rf_model.predict(X_test)
    
    # Evaluation
    accuracy = accuracy_score(y_test, rf_preds)
    conf_matrix = confusion_matrix(y_test, rf_preds)
    class_report = classification_report(y_test, rf_preds)

    print(f"Evaluation for the given vectors:\n")
    print(f'Accuracy: {accuracy:.2f}')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('Classification Report:')
    print(class_report)
    
    return rf_model

In [45]:
#hyper Parameters
rf_parameters = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Assuming X_train, y_train, X_test, and y_test are already defined
model_rf = random_forest(X_train, X_test, y_train, y_test,rf_parameters)


Best parameters found: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Evaluation for the given vectors:

Accuracy: 0.80
Confusion Matrix:
[[506  98]
 [166 530]]
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.84      0.79       604
           1       0.84      0.76      0.80       696

    accuracy                           0.80      1300
   macro avg       0.80      0.80      0.80      1300
weighted avg       0.80      0.80      0.80      1300



# Conclusion

the Random Forest ML model has better performance than the other model with a bench mark of 80% accuracy 